In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from pymedphys.level1.mudensity import *


DATA_DIRECTORY = "../../tests/data"
DELIVERY_DATA_FILEPATH = os.path.join(DATA_DIRECTORY, 'mu_density_example_arrays.npz')

def test_regression(plot=False):
    """The results of MU Density calculation should not change
    """
    regress_test_arrays = np.load(DELIVERY_DATA_FILEPATH)

    mu = regress_test_arrays['mu']
    mlc = regress_test_arrays['mlc']
    jaw = regress_test_arrays['jaw']

    cached_grid_xx, cached_grid_yy, cached_mu_density = regress_test_arrays['mu_density']

    grid_xx, grid_yy, mu_density = calc_mu_density(mu, mlc, jaw)

    assert np.all(grid_xx == cached_grid_xx)
    assert np.all(grid_yy == cached_grid_yy)
    assert np.allclose(mu_density, cached_mu_density)

    if plot:
        plt.pcolormesh(grid_xx, grid_yy, mu_density)
        plt.colorbar()
        plt.title('MU density')
        plt.xlabel('MLC direction (mm)')
        plt.ylabel('Jaw direction (mm)')
        plt.gca().invert_yaxis()


In [ ]:
test_regression(plot=True)

In [ ]:
regress_test_arrays = np.load(DELIVERY_DATA_FILEPATH)

In [ ]:
# mu

In [ ]:
# np.diff(mu)

In [ ]:
mu = regress_test_arrays['mu']
mlc = regress_test_arrays['mlc']
jaw = regress_test_arrays['jaw']

leaf_pair_widths = np.array(AGILITY_LEAF_PAIR_WIDTHS)
max_leaf_gap=400
grid_resolution=1

mu, mlc, jaw = remove_irrelevant_control_points(mu, mlc, jaw)

(
    grid_xx, grid_yy, grid_leaf_map, mlc,
    leaf_xx
) = determine_calc_grid_and_adjustments(
    mlc, jaw, leaf_pair_widths, grid_resolution)

In [ ]:
i = 3
slice_to_check = slice(i, i + 2, 1)

blocked_fraction = calc_blocked_fraction(
    leaf_xx, mlc[slice_to_check, :, :], grid_leaf_map,
    grid_yy, jaw[slice_to_check, :])

mu_density = np.sum(
    np.diff(mu[slice_to_check])[:, None, None] *
    (1 - blocked_fraction), axis=0)

plt.figure(figsize=(18,22))
plt.pcolormesh(grid_xx, grid_yy, mu_density)
plt.colorbar()
plt.title('MU density')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')
plt.axis('equal')
plt.gca().invert_yaxis()

In [ ]:
def calc_a_single_blocked_fraction(start_diffs, end_diffs,
                                   start_blocked, end_blocked):
    blocked_fraction = np.ones(np.shape(start_diffs)) * np.nan
    all_open = ~start_blocked & ~end_blocked
    blocked_fraction[all_open] = 0

    all_blocked = start_blocked & end_blocked
    blocked_fraction[all_blocked] = 1

    start_blocked_fraction = np.copy(blocked_fraction)
    end_blocked_fraction = np.copy(blocked_fraction)

    partial_blocked = start_blocked != end_blocked
    travel = np.abs(
        start_diffs[partial_blocked] -
        end_diffs[partial_blocked])

    start_partial_blocked_ref = start_blocked[partial_blocked]
    end_partial_blocked_ref = end_blocked[partial_blocked]

    start_blocked_fraction[partial_blocked & start_blocked] = np.abs(
        start_diffs[partial_blocked][start_partial_blocked_ref] /
        travel[start_partial_blocked_ref]
    )
    start_blocked_fraction[partial_blocked & end_blocked] = 0

    end_blocked_fraction[partial_blocked & end_blocked] = np.abs(
        end_diffs[partial_blocked][end_partial_blocked_ref] /
        travel[end_partial_blocked_ref]
    )
    end_blocked_fraction[partial_blocked & start_blocked] = 0

    assert np.all(~np.isnan(start_blocked_fraction))
    assert np.all(~np.isnan(end_blocked_fraction))

    return start_blocked_fraction, end_blocked_fraction

In [ ]:
x = np.linspace(-10,10, 21)
left_mlc_pos_start = [-2.3]
left_mlc_pos_end = [3.1]

right_mlc_pos_start = [0]
right_mlc_pos_end = [7.7]

left_start_diffs = x - left_mlc_pos_start
left_end_diffs = x - left_mlc_pos_end
left_start_blocked = left_start_diffs <= 0
left_end_blocked = left_end_diffs <= 0

right_start_diffs = x - right_mlc_pos_start
right_end_diffs = x - right_mlc_pos_end
right_start_blocked = right_start_diffs >= 0
right_end_blocked = right_end_diffs >= 0

start_left_blocked_fraction, end_left_blocked_fraction = calc_a_single_blocked_fraction(
    left_start_diffs, left_end_diffs, left_start_blocked, left_end_blocked)

start_right_blocked_fraction, end_right_blocked_fraction = calc_a_single_blocked_fraction(
    right_start_diffs, right_end_diffs, right_start_blocked, right_end_blocked)

In [ ]:
grid_resolution = 1

In [ ]:
np.abs(left_start_diffs) < grid_resolution/2

In [ ]:
start_left_blocked_fraction

In [ ]:
end_left_blocked_fraction

In [ ]:
start_right_blocked_fraction

In [ ]:
end_right_blocked_fraction

In [ ]:
all_start_blocked_fractions = np.concatenate([
    np.expand_dims(start_left_blocked_fraction, axis=0), 
    np.expand_dims(start_right_blocked_fraction, axis=0)
], axis=0)

all_end_blocked_fractions = np.concatenate([
    np.expand_dims(end_left_blocked_fraction, axis=0),
    np.expand_dims(end_right_blocked_fraction, axis=0)
], axis=0)

start_blocked_fraction = np.max(all_start_blocked_fractions, axis=0)
end_blocked_fraction = np.max(all_end_blocked_fractions, axis=0)

blocked_fraction = start_blocked_fraction + end_blocked_fraction
blocked_fraction[blocked_fraction > 1] = 1

In [ ]:
blocked_fraction

In [ ]:
plt.plot(x, 1 - blocked_fraction, '-o')

In [ ]:
def a_single_mlc_pair(x, left_mlc_pos_start, left_mlc_pos_end, right_mlc_pos_start, right_mlc_pos_end):
    left_start_diffs = x - left_mlc_pos_start
    left_end_diffs = x - left_mlc_pos_end
    left_start_blocked = left_start_diffs <= 0
    left_end_blocked = left_end_diffs <= 0

    right_start_diffs = x - right_mlc_pos_start
    right_end_diffs = x - right_mlc_pos_end
    right_start_blocked = right_start_diffs >= 0
    right_end_blocked = right_end_diffs >= 0

    start_left_blocked_fraction, end_left_blocked_fraction = calc_a_single_blocked_fraction(
        left_start_diffs, left_end_diffs, left_start_blocked, left_end_blocked)

    start_right_blocked_fraction, end_right_blocked_fraction = calc_a_single_blocked_fraction(
        right_start_diffs, right_end_diffs, right_start_blocked, right_end_blocked)
    
    all_start_blocked_fractions = np.concatenate([
        np.expand_dims(start_left_blocked_fraction, axis=0), 
        np.expand_dims(start_right_blocked_fraction, axis=0)
    ], axis=0)

    all_end_blocked_fractions = np.concatenate([
        np.expand_dims(end_left_blocked_fraction, axis=0),
        np.expand_dims(end_right_blocked_fraction, axis=0)
    ], axis=0)

    start_blocked_fraction = np.max(all_start_blocked_fractions, axis=0)
    end_blocked_fraction = np.max(all_end_blocked_fractions, axis=0)

    blocked_fraction = start_blocked_fraction + end_blocked_fraction
    blocked_fraction[blocked_fraction > 1] = 1
    
    return blocked_fraction



In [ ]:
left_mlc_pos_start = -4
left_mlc_pos_end = 1

right_mlc_pos_start = -3.99
right_mlc_pos_end = 1.01

In [ ]:
x_coarse = np.linspace(-10, 10, 21)

blocked_fraction_coarse = a_single_mlc_pair(
    x_coarse, left_mlc_pos_start, left_mlc_pos_end, right_mlc_pos_start, right_mlc_pos_end)

plt.plot(x_coarse, 1 - blocked_fraction_coarse, '-o')

In [ ]:
blocked_fraction_coarse

In [ ]:
x_fine = np.linspace(-10, 10, 2001)

blocked_fraction_fine = a_single_mlc_pair(
    x_fine, left_mlc_pos_start, left_mlc_pos_end, right_mlc_pos_start, right_mlc_pos_end)

plt.plot(x_fine, 1 - blocked_fraction_fine, '-o')

In [ ]:
reference = np.argmin(np.abs(x_fine[None,:] - x_coarse[:,None]), axis=0)

average_blocked_fine = []
for i, x_val in enumerate(x_coarse):
    average_blocked_fine.append(
        np.mean(blocked_fraction_fine[reference == i]))
    
average_blocked_fine = np.array(average_blocked_fine)

In [ ]:
plt.plot(x_coarse, 1 - average_blocked_fine, '-o')
plt.plot(x_coarse, 1 - blocked_fraction_coarse, '-o')

In [ ]:
diff = (
    (1 - average_blocked_fine) -
    (1 - blocked_fraction_coarse)
)

# plt.plot(x_coarse, 1 - average_blocked_fine, '-o')
plt.plot(x_coarse, diff, '-o')
plt.plot(x_coarse, np.zeros_like(x_coarse), '-')

In [ ]:
test = np.linspace(0, 1, 100)

xx, yy = np.meshgrid(test, test)

In [ ]:
zz = np.ones_like(xx) / np.size(xx)
np.sum(zz)

In [ ]:
result = []
for value in test:
    result.append(np.sum(zz[(xx >= value) & (yy >= value)]))
    
plt.plot(test, result)
plt.plot(test, (test-1)**2)

In [ ]:
#### Proposed new method prototype

In [ ]:
grid_spacing = 1

In [ ]:
grid = {
    'mlc': np.arange(0.5,3, grid_spacing),
    'jaw': np.arange(0.5,5, grid_spacing)
}

In [ ]:
positions = {
    'mlc': {
        -1: (2, 0), # right
        1: (1.66667, 0) # left
    },
    'jaw': {
        1: (0, 3), # top
        -1: (1, 5) # bot
    }
}

In [ ]:
def calc_blocked_t(travel_diff, grid_spacing):
    blocked_t = np.ones_like(travel_diff) * np.nan

    fully_blocked = travel_diff <= -grid_spacing/2
    fully_open = travel_diff >= grid_spacing/2
    blocked_t[fully_blocked] = 1
    blocked_t[fully_open] = 0

    transient = ~fully_blocked & ~fully_open

    blocked_t[transient] = (
        (-travel_diff[transient] + grid_spacing/2) / 
        grid_spacing)

    assert np.all(~np.isnan(blocked_t))
    
    return blocked_t

In [ ]:
blocked_by_device = {}

for device, value in positions.items():
    blocked_by_device[device] = dict()
    
    for multiplier, (start, end) in value.items():
        travel = np.linspace(start, end)
        travel_diff = multiplier * (grid[device][:,None] - travel[None,:])
        
        blocked_by_device[device][multiplier] = calc_blocked_t(
            travel_diff, grid_spacing)

In [ ]:
plt.plot(1 - blocked_by_device['mlc'][-1][1])
plt.ylim([-0.1,1.1])

In [ ]:
plt.plot(1 - blocked_by_device['mlc'][1][1])
plt.ylim([-0.1,1.1])

In [ ]:
plt.plot(1 - blocked_by_device['jaw'][1][1])
plt.ylim([-0.1,1.1])

In [ ]:
plt.plot(1 - blocked_by_device['jaw'][-1][1])
plt.ylim([-0.1,1.1])

In [ ]:
device_open = {}

for device, value in blocked_by_device.items():
    device_sum = np.sum(np.concatenate([
        np.expand_dims(blocked, axis=0)
        for _, blocked in value.items()
    ], axis=0), axis=0)
    
    device_open[device] = 1 - device_sum
    
# device_sums

In [ ]:
plt.plot(device_open['mlc'][1])
plt.plot(
    1 - (
        blocked_by_device['mlc'][1][1] +
        blocked_by_device['mlc'][-1][1])
)
plt.ylim([-0.1,1.1])

In [ ]:
plt.plot(device_open['jaw'][1])
plt.plot(
    1 - (
        blocked_by_device['jaw'][1][1] +
        blocked_by_device['jaw'][-1][1])
)
plt.ylim([-0.1,1.1])

In [ ]:
open_t = device_open['mlc'][:, None, :] * device_open['jaw'][None, :, :]
np.shape(open_t)

In [ ]:
plt.plot(open_t[1,1,:])
plt.plot(device_open['jaw'][1] * device_open['mlc'][1])
plt.ylim([-0.1,1.1])

In [ ]:
open_fraction = np.mean(open_t, axis=2)
open_fraction